In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [5]:
temp_df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
df = temp_df.iloc[:10000]

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.drop_duplicates(inplace=True)

/tmp/ipython-input-3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [9]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [10]:
df['review'] = df['review'].apply(remove_tags)

/tmp/ipython-input-2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [11]:
df['review'] = df['review'].apply(lambda x:x.lower())

/tmp/ipython-input-740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [14]:
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('punkt_tab')
import nltk
nltk.download('stopwords')



sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/tmp/ipython-input-391691923.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [15]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [17]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 73.5 MB/s eta 0:00:00


In [21]:
import gensim
from nltk import sent_tokenize

from gensim.utils import simple_preprocess

In [22]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [23]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [24]:
model.build_vocab(story)

In [25]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5849736, 6186875)

In [26]:
len(model.wv.index_to_key)

31845

In [27]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [28]:
document_vector(df['review'].values[0])

array([-0.04175934,  0.3652772 ,  0.21758915, -0.07045204, -0.05425413,
       -0.533749  ,  0.15586926,  0.7826111 , -0.21977505, -0.05635104,
       -0.07266986, -0.52166146,  0.03682303,  0.18179287,  0.05557284,
       -0.20794754,  0.130838  , -0.53535396,  0.0228711 , -0.6647558 ,
        0.1728124 ,  0.10226602,  0.20543863, -0.17781025, -0.26005617,
       -0.13273396, -0.25365677, -0.29656106, -0.35118276,  0.02421406,
        0.4179704 ,  0.03204715,  0.03657974, -0.26641792, -0.22704089,
        0.2642776 ,  0.07302362, -0.3616442 , -0.24165596, -0.70150876,
        0.13386281, -0.30186975, -0.25184157,  0.03403936,  0.39390227,
       -0.25196093, -0.40925384, -0.16551496,  0.20466347,  0.3100101 ,
        0.13702029, -0.2756101 , -0.15520787, -0.14423539, -0.46978524,
        0.06554189,  0.36425877,  0.08060665, -0.37685442,  0.04942123,
       -0.00354237,  0.18484858, -0.16549553, -0.0153465 , -0.40532747,
        0.3615216 , -0.04897582,  0.22360659, -0.56726885,  0.26

In [29]:
from tqdm import tqdm

In [30]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [08:28<00:00, 19.62it/s]


In [31]:
X = np.array(X)

In [32]:
X[0]

array([-0.04175934,  0.3652772 ,  0.21758915, -0.07045204, -0.05425413,
       -0.533749  ,  0.15586926,  0.7826111 , -0.21977505, -0.05635104,
       -0.07266986, -0.52166146,  0.03682303,  0.18179287,  0.05557284,
       -0.20794754,  0.130838  , -0.53535396,  0.0228711 , -0.6647558 ,
        0.1728124 ,  0.10226602,  0.20543863, -0.17781025, -0.26005617,
       -0.13273396, -0.25365677, -0.29656106, -0.35118276,  0.02421406,
        0.4179704 ,  0.03204715,  0.03657974, -0.26641792, -0.22704089,
        0.2642776 ,  0.07302362, -0.3616442 , -0.24165596, -0.70150876,
        0.13386281, -0.30186975, -0.25184157,  0.03403936,  0.39390227,
       -0.25196093, -0.40925384, -0.16551496,  0.20466347,  0.3100101 ,
        0.13702029, -0.2756101 , -0.15520787, -0.14423539, -0.46978524,
        0.06554189,  0.36425877,  0.08060665, -0.37685442,  0.04942123,
       -0.00354237,  0.18484858, -0.16549553, -0.0153465 , -0.40532747,
        0.3615216 , -0.04897582,  0.22360659, -0.56726885,  0.26

In [33]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [34]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [37]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7701552328492739

In [41]:
from sklearn.svm import SVC

In [42]:
svm=SVC()
svm.fit(X_train,y_train)

SVC()

In [43]:
svc_pred = svm.predict(X_test)

In [44]:
accuracy_score(y_test,svc_pred)

0.800701051577366